In [12]:
!pip install holidays

     |████████████████████████████████| 114 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 732 kB 1.2 MB/s eta 0:00:01
  Created wheel for holidays: filename=holidays-0.10.3-py3-none-any.whl size=115855 sha256=62dfd49ec05cdef13b3dffed77b0ff22a99cbfb5af4f57953a42b463aa70794f
  Stored in directory: /Users/kennethcheng/Library/Caches/pip/wheels/a2/97/85/fd3fe9e0f96120582caa3470bde6652f826b4e5842e3cce65c
  Created wheel for pymeeus: filename=PyMeeus-0.3.7-py3-none-any.whl size=702876 sha256=ebfa3005e306df8413a3e1d2369b5791df9a9f0af1dac8bb3d1b30af2e91cc27
  Stored in directory: /Users/kennethcheng/Library/Caches/pip/wheels/5a/68/50/d989a005ecd4f58a7922bede25ff7e391d66395a3090acf97a
Successfully built holidays pymeeus


In [15]:
!pip install xgboost

     |████████████████████████████████| 1.2 MB 1.2 MB/s eta 0:00:01


In [16]:
!pip install catboost


     |████████████████████████████████| 11.5 MB 169 kB/s eta 0:00:01
     |████████████████████████████████| 13.1 MB 1.2 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=0bb14a2200a257ee63f33dfe1644271afe0e5fdb87cfb8fb19b654e9e5bc861b
  Stored in directory: /Users/kennethcheng/Library/Caches/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


In [17]:
!pip install hyperopt

     |████████████████████████████████| 965 kB 1.1 MB/s eta 0:00:01


In [19]:
!brew install libomp

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 3 taps (homebrew/core, homebrew/cask and homebrew/cask-fonts).
==> New Formulae
aerc                       libaio                     ncspot
arturo                     libbsd                     nicotine-plus
asroute                    libcap                     node@14
aws-rotate-key             libdmx                     openjdk@8
bit-git                    libdrm                     or-tools
blaze                      libfontenc                 overdrive
box2d                      libfs                      ox
bpytop                     libfuse                    pdm
cairomm@1.14               libgccjit                  phive
cargo-edit                 libhandy                   php@7.4
cfn-flip                   libice                     pickle
cfn-format                 libmnl                     postgresql@12
chars                      libnetfilter-queue         protoc-gen-go-grpc
checkov                    libnfnetlink    

In [20]:
import numpy as np
import pandas as pd
from datetime import date
import holidays
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import catboost as ctb
from hyperopt import hp
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [21]:
train_df = pd.read_csv("MSBD5001/input/train.csv", parse_dates = ['date'])
test_df = pd.read_csv("MSBD5001/input/test.csv", parse_dates = ['date'])

In [ ]:
# rainfall_2017 = pd.read_csv("MSBD5001/input/rainfall_2017.csv")
# rainfall_2018 = pd.read_csv("MSBD5001/input/rainfall_2018.csv")

In [ ]:
# rainfall_2017 = rainfall_2017.replace('Trace',0)
# rainfall_2017 = rainfall_2017.fillna(0)
# rainfall_2018 = rainfall_2018.replace('Trace',0)
# rainfall_2018 = rainfall_2018.fillna(0)

In [ ]:
# def check_rainfall(date):
#     if date.year == 2017:
#         return rainfall_2017.iat[date.day-1,date.month]
#     else:
#         return rainfall_2018.iat[date.day-1,date.month]
    


In [22]:
hk_holidays = holidays.HongKong(years=[2017,2018])

def check_holiday(date):
    if date in hk_holidays:
        return 1
    else:
        return 0

def check_weekday(date):
    if date < 5:
        return 1
    else:
        return 0

def extract_date(df,column):
    df[column+"_year"] = df[column].apply(lambda x: x.year)
    df[column+"_month"] = df[column].apply(lambda x: x.month)
    df[column+"_day"] = df[column].apply(lambda x: x.day)
    df[column+"_hour"] = df[column].apply(lambda x: x.hour)
    df[column+"_weekday"] = df[column].dt.weekday
#     df[column+"_weekdayind"] = df[column].dt.weekday.apply(check_weekday)
    df[column+"_holiday"] = df[column].apply(check_holiday)
#     df[column+"_rainfall"] = df[column].apply(check_rainfall)

In [23]:
extract_date(train_df,'date')
extract_date(test_df,'date')

In [24]:
# train_df['date_rainfall'] = train_df['date_rainfall'].astype('float64')
# test_df['date_rainfall'] = test_df['date_rainfall'].astype('float64')

In [25]:
train_df.head(100)

,id,date,speed,date_year,date_month,date_day,date_hour,date_weekday,date_holiday
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,6,0
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,6,0
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,6,0
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,6,0
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,6,0
...,...,...,...,...,...,...,...,...,...
95,95,2017-04-01 23:00:00,46.235164,2017,4,1,23,5,0
96,96,2017-05-01 00:00:00,47.620550,2017,5,1,0,0,1
97,97,2017-05-01 01:00:00,49.338859,2017,5,1,1,0,1
98,98,2017-05-01 02:00:00,47.116882,2017,5,1,2,0,1


In [26]:
test_df.head(100)

,id,date,date_year,date_month,date_day,date_hour,date_weekday,date_holiday
0,0,2018-01-01 02:00:00,2018,1,1,2,0,1
1,1,2018-01-01 05:00:00,2018,1,1,5,0,1
2,2,2018-01-01 07:00:00,2018,1,1,7,0,1
3,3,2018-01-01 08:00:00,2018,1,1,8,0,1
4,4,2018-01-01 10:00:00,2018,1,1,10,0,1
...,...,...,...,...,...,...,...,...
95,95,2018-10-01 08:00:00,2018,10,1,8,0,1
96,96,2018-10-01 10:00:00,2018,10,1,10,0,1
97,97,2018-10-01 17:00:00,2018,10,1,17,0,1
98,98,2018-10-01 18:00:00,2018,10,1,18,0,1


In [27]:
X = train_df.drop(['id', 'date', 'speed'], axis=1)
y = train_df['speed']

test_X = test_df.drop(['id', 'date'], axis=1)

In [28]:
reg = LinearRegression().fit(X, y)
reg.score(X,y), mean_squared_error(reg.predict(X), y)

(0.17162828027727328, 152.615277119845)

In [29]:
regr = RandomForestRegressor(max_depth=20, random_state=0)
regr.fit(X, y)
regr.score(X,y), mean_squared_error(regr.predict(X), y)

(0.9827200652469894, 3.1835732294509995)

In [30]:
# Current best

xgbr = XGBRegressor()
xgbr.fit(X, y)

xgbr.score(X,y), mean_squared_error(xgbr.predict(X), y)

(0.9392051934072398, 11.200546849555748)

In [31]:
ctbr = ctb.CatBoostRegressor()
ctbr.fit(X, y)

ctbr.score(X,y), mean_squared_error(ctbr.predict(X), y)

Learning rate set to 0.062437
0:	learn: 12.9497359	total: 65.8ms	remaining: 1m 5s
1:	learn: 12.3541319	total: 70ms	remaining: 34.9s
2:	learn: 11.8073638	total: 74.6ms	remaining: 24.8s
3:	learn: 11.2935192	total: 79.1ms	remaining: 19.7s
4:	learn: 10.8183354	total: 83.4ms	remaining: 16.6s
5:	learn: 10.3832786	total: 87.9ms	remaining: 14.6s
6:	learn: 10.0068048	total: 92.1ms	remaining: 13.1s
7:	learn: 9.6330309	total: 96.8ms	remaining: 12s
8:	learn: 9.2928872	total: 101ms	remaining: 11.1s
9:	learn: 8.9822260	total: 106ms	remaining: 10.5s
10:	learn: 8.6990732	total: 110ms	remaining: 9.89s
11:	learn: 8.4493168	total: 115ms	remaining: 9.49s
12:	learn: 8.2204386	total: 120ms	remaining: 9.1s
13:	learn: 8.0054941	total: 124ms	remaining: 8.75s
14:	learn: 7.8015988	total: 129ms	remaining: 8.45s
15:	learn: 7.6120913	total: 131ms	remaining: 8.06s
16:	learn: 7.4385130	total: 134ms	remaining: 7.72s
17:	learn: 7.2851104	total: 136ms	remaining: 7.43s
18:	learn: 7.1470551	total: 138ms	remaining: 7.14s
1

(0.9140476361010533, 15.835455898873818)

In [32]:
# XGB parameters
xgb_reg_params = {
    'max_depth': hp.choice('max_depth', np.arange(10, 100, dtype=int)),
    'min_child_weight': hp.quniform ('min_child_weight', 1, 50, 1),
    'subsample': hp.uniform ('subsample', 0.1, 1),
    'n_estimators' : hp.choice('n_estimators', np.arange(100, 10000, 100, dtype=int)),
    'learning_rate' : hp.quniform('learning_rate', 0.025, 1.0, 0.025),
    'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.2, 1, 0.05)
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

# CTB parameters
ctb_reg_params = {
    'learning_rate' : hp.quniform('learning_rate', 0.025, 1.0, 0.025),
    'max_depth': hp.choice('max_depth', np.arange(1, 16, dtype=int)),
    'colsample_bylevel': hp.quniform('colsample_bylevel', 0.2, 1, 0.05),
    'n_estimators' : hp.choice('n_estimators', np.arange(100, 10000, 100, dtype=int)),
    'eval_metric':       'RMSE',
}
ctb_fit_params = {
    'early_stopping_rounds': 10,
    'verbose': False
}
ctb_para = dict()
ctb_para['reg_params'] = ctb_reg_params
ctb_para['fit_params'] = ctb_fit_params
ctb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

In [33]:
class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def ctb_reg(self, para):
        reg = ctb.CatBoostRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

obj = HPOpt(X_train, X_test, y_train, y_test)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [01:17<00:00,  1.29trial/s, best loss: 4.016086154397904]


In [35]:
xgb_opt

({'colsample_bytree': 0.8,
  'gamma': 1.0,
  'learning_rate': 0.07500000000000001,
  'max_depth': 31,
  'min_child_weight': 8.0,
  'n_estimators': 90,
  'subsample': 0.9550623611066004},
 <hyperopt.base.Trials at 0x1238ca5b0>)

In [36]:
xgbr = XGBRegressor(**xgb_opt[0])
xgbr.fit(X, y)

xgbr.score(X,y), mean_squared_error(xgbr.predict(X), y)

(0.9451699423420417, 10.101629793406397)

In [37]:
test_df['speed'] = xgbr.predict(test_df.drop(['id', 'date'], axis=1))

In [38]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# obj = HPOpt(X_train, X_test, y_train, y_test)

# ctb_opt = obj.process(fn_name='ctb_reg', space=ctb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

In [39]:
# ctb_opt

In [40]:
# ctbr =  ctb.CatBoostRegressor(**ctb_opt[0])
# ctbr.fit(X, y)

# ctbr.score(X,y), mean_squared_error(ctbr.predict(X), y)

In [41]:
# test_df['speed'] = ctbr.predict(test_df.drop(['id', 'date'], axis=1))

In [42]:
sample_submission = test_df[['id', 'speed']]
sample_submission.to_csv(r'MSBD5001/output//msbd5001fall2020_Sample_Submission.csv', index = False)